To create a simple API with JWT authentication in **Node.js**, you can use the `express` framework along with the `jsonwebtoken` library. Below is a guide on how to set up a Node.js API with JWT authentication.

### 1. Install Required Libraries

First, you'll need to install the necessary libraries using npm:

```bash
npm init -y
npm install express jsonwebtoken bcryptjs dotenv
```

- `express`: A web framework for building APIs.
- `jsonwebtoken`: For creating and verifying JWT tokens.
- `bcryptjs`: For hashing passwords.
- `dotenv`: For managing environment variables.

### 2. Create Your Project Files

Let's organize the project with the following structure:

```
/jwt-api
    |-- .env
    |-- app.js
    |-- controllers/
        |-- authController.js
    |-- middleware/
        |-- authMiddleware.js
    |-- models/
        |-- user.js
```

### 3. Configure Environment Variables

Create a `.env` file at the root of the project and add the following:

```env
JWT_SECRET=your_jwt_secret_key
JWT_EXPIRATION=1h
```

### 4. Set Up the Express Server

Create a file named `app.js` to set up the Express server.

```javascript
// app.js
const express = require('express');
const dotenv = require('dotenv');
const authRoutes = require('./controllers/authController');

dotenv.config();  // Load environment variables

const app = express();
app.use(express.json());  // Middleware to parse JSON bodies

// Use routes for authentication
app.use('/api/auth', authRoutes);

// Home route
app.get('/', (req, res) => {
    res.send('Welcome to the Node.js JWT API');
});

// Start the server
const port = process.env.PORT || 5000;
app.listen(port, () => {
    console.log(`Server running on port ${port}`);
});
```

### 5. Create the User Model

In `models/user.js`, define a simple User model. For simplicity, we'll use an in-memory "database" in this example.

```javascript
// models/user.js
const bcrypt = require('bcryptjs');

let users = [];  // In-memory user store

// Simulate finding a user by username
function findUserByUsername(username) {
    return users.find(user => user.username === username);
}

// Simulate creating a user
function createUser(username, password) {
    const hashedPassword = bcrypt.hashSync(password, 8);
    const user = { username, password: hashedPassword };
    users.push(user);
    return user;
}

module.exports = { findUserByUsername, createUser };
```

### 6. Authentication Controller

Now, create the authentication controller that handles login and token generation in `controllers/authController.js`.

```javascript
// controllers/authController.js
const express = require('express');
const jwt = require('jsonwebtoken');
const bcrypt = require('bcryptjs');
const { findUserByUsername, createUser } = require('../models/user');
const { JWT_SECRET, JWT_EXPIRATION } = process.env;
const router = express.Router();

// Register user
router.post('/register', (req, res) => {
    const { username, password } = req.body;
    
    if (!username || !password) {
        return res.status(400).json({ message: 'Username and password are required' });
    }

    const existingUser = findUserByUsername(username);
    if (existingUser) {
        return res.status(400).json({ message: 'User already exists' });
    }

    const newUser = createUser(username, password);
    res.status(201).json({ message: 'User created', user: newUser });
});

// Login and generate token
router.post('/login', (req, res) => {
    const { username, password } = req.body;

    const user = findUserByUsername(username);
    if (!user) {
        return res.status(404).json({ message: 'User not found' });
    }

    // Check password
    if (!bcrypt.compareSync(password, user.password)) {
        return res.status(400).json({ message: 'Invalid password' });
    }

    // Generate JWT token
    const token = jwt.sign({ username: user.username }, JWT_SECRET, {
        expiresIn: JWT_EXPIRATION,
    });

    res.json({ message: 'Login successful', token });
});

module.exports = router;
```

### 7. Authorization Middleware

Create middleware for protected routes in `middleware/authMiddleware.js`.

```javascript
// middleware/authMiddleware.js
const jwt = require('jsonwebtoken');
const { JWT_SECRET } = process.env;

// Middleware to check JWT token
function authenticateToken(req, res, next) {
    const token = req.header('Authorization')?.split(' ')[1];  // Get token from Authorization header

    if (!token) {
        return res.status(403).json({ message: 'Access denied. No token provided.' });
    }

    jwt.verify(token, JWT_SECRET, (err, user) => {
        if (err) {
            return res.status(403).json({ message: 'Invalid token' });
        }

        req.user = user;  // Attach the user information to the request
        next();
    });
}

module.exports = authenticateToken;
```

### 8. Create a Protected Route

Now, let's create a protected route to test the JWT authentication.

Update `app.js` to include the protected route:

```javascript
// app.js (continued)
const authenticateToken = require('./middleware/authMiddleware');

// A protected route that requires authentication
app.get('/api/protected', authenticateToken, (req, res) => {
    res.json({ message: `Hello, ${req.user.username}! This is a protected route.` });
});
```

### 9. Running the Application

1. Start the server:

   ```bash
   node app.js
   ```

2. **Register a User**: You can use Postman or `curl` to register a new user.

   ```bash
   curl -X POST http://localhost:5000/api/auth/register -H "Content-Type: application/json" -d '{"username": "testuser", "password": "password123"}'
   ```

3. **Login and Get Token**: After registering, login to get the JWT token:

   ```bash
   curl -X POST http://localhost:5000/api/auth/login -H "Content-Type: application/json" -d '{"username": "testuser", "password": "password123"}'
   ```

   The response will contain an access token:

   ```json
   {
     "message": "Login successful",
     "token": "<your_token_here>"
   }
   ```

4. **Access Protected Route**: Use the token to access a protected route:

   ```bash
   curl -X GET http://localhost:5000/api/protected -H "Authorization: Bearer <your_token_here>"
   ```

   If the token is valid, you'll get a response like:

   ```json
   {
     "message": "Hello, testuser! This is a protected route."
   }
   ```

### Summary

This Node.js app uses **Express** for routing, **jsonwebtoken** for JWT creation and verification, and **bcryptjs** for password hashing. The application features:

- User registration (stores hashed passwords).
- Login (generates JWT).
- A protected route that only allows access with a valid JWT token.

This setup provides a secure API where sensitive routes are protected using JWT authentication.